In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string
from sklearn.metrics import accuracy_score
# Download NLTK stopwords (only once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, word_tokenize

# If not done already, download necessary NLTK resources
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import random
import warnings
warnings.filterwarnings("ignore")
nltk.download('wordnet')
from nltk.corpus import wordnet
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from transformers import BertForSequenceClassification, get_scheduler
from torch.optim import AdamW
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Initialize
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words.remove('not')  # Keep 'not' for better negation handling

# POS tag converter
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Clean + lemmatize function
def clean_and_lemmatize(text):
    if pd.isnull(text):
        return ""
    text = str(text)
    text = re.sub(r'\bRT\b', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub('[^a-zA-Z\s]', ' ', text)
    text = text.lower()

    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in tagged_tokens
        if word not in stop_words
    ]
    return ' '.join(lemmatized_words)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
dataset_path='/content/drive/MyDrive/FreelanceProjects/StudentSelection'
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    hidden_dropout_prob=0.3
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Re-initialize the same model architecture
os.chdir(dataset_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.load_state_dict(torch.load("internship_selection_model.pt", map_location=torch.device('cpu')))
model.to('cpu')
model.eval()
print("✅ Model loaded from 'internship_selection_model.pt'")


✅ Model loaded from 'internship_selection_model.pt'


In [ ]:
sample_2 = {
    'town': 'St. Johns',
    'identify_as': 'Male',
    'specify_below': ' ',
    "why_internship": "I have an interest in tech and hope to have a tech profession in the future. This opportunity will help me solidify that dream.",
    "tech_experience": "I am able to research online, use Google Sheets, and word for school work. I am also play games on PlayStation.",
    "non_tech_experience": "I am a dedicated volunteer in my community. I come from a larger size family where team work makes the dream work where I am the second eldest so a lot of responsibilities fall onto me. I have played on many all star soccer teams and school teams all throughout my time in school and I am personable and approachable and eager to learn.",
    "goals": "I will be attending MUN in September for engineering.",
    "other_comments": "I am excited for this opportunity."
}


In [ ]:
# Define the full list of fields you want to clean
fields_to_clean = [
    'town', 'identify_as', 'specify_below',
    'why_internship', 'tech_experience', 'non_tech_experience', 'goals', 'other_comments'
]

# Clean all selected fields in the sample
cleaned_sample = {
    key: clean_and_lemmatize(value) if key in fields_to_clean else value
    for key, value in sample_2.items()
}

# Combine all fields into one text string for the model
def combine_fields(entry):
    return " ".join([entry.get(col, "") for col in fields_to_clean])

test_samples = [combine_fields(cleaned_sample)]

# Tokenize and move to device
inputs = tokenizer(test_samples, return_tensors="pt", padding=True, truncation=True, max_length=256)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Run model prediction
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

# Display result
for i, raw_text in enumerate(test_samples):
    label = "✅ SELECTED" if preds[i].item() == 1 else "❌ NOT SELECTED"
    confidence = probs[i][preds[i]].item()
    print(f"\n➡️ Prediction: {label} (Confidence: {confidence:.2f})\n")



➡️ Prediction: ❌ NOT SELECTED (Confidence: 0.89)

